In [1]:
import sys, os
import numpy as np
import pandas as pd
import pyproj
import folium

In [2]:
culture1 = pd.read_csv("POI_Ori/문화/fulldata_03_06_01_P_공연장.csv", encoding='cp949', usecols=['개방서비스명', '영업상태명', '소재지전체주소', '좌표정보(x)', '좌표정보(y)'])
culture2 = pd.read_csv("POI_Ori/문화/fulldata_03_07_01_P_관광궤도업.csv", encoding='cp949', usecols=['개방서비스명', '영업상태명', '소재지전체주소', '좌표정보(x)', '좌표정보(y)'])
culture3 = pd.read_csv("POI_Ori/문화/fulldata_03_07_04_P_국제회의시설업.csv", encoding='cp949', usecols=['개방서비스명', '영업상태명', '소재지전체주소', '좌표정보(x)', '좌표정보(y)'])
culture4 = pd.read_csv("POI_Ori/문화/fulldata_03_07_05_P_박물관,미술관.csv", encoding='cp949', usecols=['개방서비스명', '영업상태명', '소재지전체주소', '좌표정보(x)', '좌표정보(y)'])
culture5 = pd.read_csv("POI_Ori/문화/fulldata_03_07_10_P_전문휴양업.csv", encoding='cp949', usecols=['개방서비스명', '영업상태명', '소재지전체주소', '좌표정보(x)', '좌표정보(y)'])
culture6 = pd.read_csv("POI_Ori/문화/fulldata_03_07_11_P_전통사찰.csv", encoding='cp949', usecols=['개방서비스명', '영업상태명', '소재지전체주소', '좌표정보(x)', '좌표정보(y)'])
culture7 = pd.read_csv("POI_Ori/문화/fulldata_03_07_12_P_종합유원시설업.csv", encoding='cp949', usecols=['개방서비스명', '영업상태명', '소재지전체주소', '좌표정보(x)', '좌표정보(y)'])
culture8 = pd.read_csv("POI_Ori/문화/fulldata_03_07_13_P_종합휴양업.csv", encoding='cp949', usecols=['개방서비스명', '영업상태명', '소재지전체주소', '좌표정보(x)', '좌표정보(y)'])
culture9 = pd.read_csv("POI_Ori/문화/fulldata_03_11_01_P_관광숙박업.csv", encoding='cp949', usecols=['개방서비스명', '영업상태명', '소재지전체주소', '좌표정보(x)', '좌표정보(y)'])
culture10 = pd.read_csv("POI_Ori/문화/fulldata_03_11_02_P_관광펜션업.csv", encoding='cp949', usecols=['개방서비스명', '영업상태명', '소재지전체주소', '좌표정보(x)', '좌표정보(y)'])
culture11 = pd.read_csv("POI_Ori/문화/fulldata_03_11_05_P_자동차야영장업.csv", encoding='cp949', usecols=['개방서비스명', '영업상태명', '소재지전체주소', '좌표정보(x)', '좌표정보(y)'])
culture12 = pd.read_csv("POI_Ori/문화/fulldata_03_11_07_P_일반야영장업.csv", encoding='cp949', usecols=['개방서비스명', '영업상태명', '소재지전체주소', '좌표정보(x)', '좌표정보(y)'])
culture13 = pd.read_csv("POI_Ori/문화/fulldata_03_13_02_P_영화상영관.csv", encoding='cp949', usecols=['개방서비스명', '영업상태명', '소재지전체주소', '좌표정보(x)', '좌표정보(y)'])

culture_num = ['culture1', 'culture2', 'culture3', 'culture4', 'culture5', 'culture6', 'culture7', 'culture8', 'culture9', 'culture10', 'culture11', 'culture12', 'culture13']
culture_name = ['공연장', '관광궤도업', '국제회의시설업', '박물관,미술관', '전문휴양업', '전통사찰', '종합유원시설업', '종합휴양업', '관광숙박업', '관광펜션업', '자동차야영자업', '일반야영장업', '영화상영관']

In [3]:
def project_array(coord, p1_type, p2_type):
    """
    좌표계 변환 함수
    - coord: x, y 좌표 정보가 담긴 NumPy Array
    - p1_type: 입력 좌표계 정보 ex) epsg:5179
    - p2_type: 출력 좌표계 정보 ex) epsg:4326
    """
    p1 = pyproj.Proj(init=p1_type)
    p2 = pyproj.Proj(init=p2_type)
    fx, fy = pyproj.transform(p1, p2, coord[:, 3], coord[:, 4])
    
    return np.dstack([fx, fy])[0]

def data_processing(name):
    name.drop(name[(name['영업상태명'] == '폐업')].index, inplace=True)
    name = name[name['영업상태명'].notna()]
    name = name[~name['영업상태명'].str.contains('정지')]
    name = name[name['소재지전체주소'].str.contains('서울특별시') & name['소재지전체주소'].notna()]
    
    name['좌표정보(x)'] = pd.to_numeric(name['좌표정보(x)'], errors="coerce")
    name['좌표정보(y)'] = pd.to_numeric(name['좌표정보(y)'], errors="coerce")
    name = name.dropna(axis=0)
    name.index = range(len(name))
    coord = np.array(name)

    p1_type = "epsg:2097"
    p2_type = "epsg:4326"

    result = project_array(coord, p1_type, p2_type)
    print("okay")
    name['경도'] = result[:, 0]
    name['위도'] = result[:, 1]
    
    return name

In [4]:
culture1

,개방서비스명,영업상태명,소재지전체주소,좌표정보(x),좌표정보(y)
0,공연장,휴업,서울특별시 종로구 동숭동 1-58,200153.322798,453332.518876
1,공연장,휴업,전라북도 남원시 어현동 37-140,235576.052370,211603.276473
2,공연장,휴업,전라북도 군산시 나운동 790-3,172373.756140,274055.070334
3,공연장,휴업,전라북도 완주군 구이면 백여리 342번지,209847.557584,239472.476412
4,공연장,휴업,강원특별자치도 속초시 조양동 1545-1,338575.492428,522063.941893
...,...,...,...,...,...
2662,공연장,영업/정상,서울특별시 영등포구 영등포동3가 2-0번지,191960.889845,446273.110783
2663,공연장,영업/정상,서울특별시 영등포구 영등포동2가 175-0번지,191952.897387,446340.637133
2664,공연장,영업/정상,서울특별시 영등포구 영등포동2가 175-0번지,191952.897387,446340.637133
2665,공연장,영업/정상,서울특별시 중구 예장동 8-3번지,198817.114278,450514.417668


In [5]:
culture1 = data_processing(culture1)
culture2 = data_processing(culture2)
culture3 = data_processing(culture3)
culture4 = data_processing(culture4)
culture5 = data_processing(culture5)
culture6 = data_processing(culture6)
culture7 = data_processing(culture7)
culture8 = data_processing(culture8)
culture9 = data_processing(culture9)
culture10 = data_processing(culture10)
culture11 = data_processing(culture11)
culture12 = data_processing(culture12)
culture13 = data_processing(culture13)

c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\User\AppData\Local\Temp\ipykernel_13876\1567479730.py:10: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.

okay
okay
okay
okay
okay
okay
okay
okay
okay
okay
okay
okay
okay


c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\User\AppData\Local\Temp\ipykernel_13876\1567479730.py:10: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.